<a href="https://colab.research.google.com/github/manoharpavuluri/AI_FTCIntoTheDeep_RAG/blob/main/AI_FTCIntoTheDeep_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!pip install openai --quiet
!pip install langchain --quiet
!pip install cohere --quiet
!pip install tiktoken --quiet
!pip install langchain_community --quiet
!pip install --upgrade langchain-core


In [3]:
import os

In [53]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['COHERE_API_KEY'] = userdata.get('COHERE_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')



In [5]:
def pretty_print_docs(docs):
  print(
      f"\n{'-' * 100}\n".join(
          [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
      )
  )

In [6]:
!pip install pypdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.7 MB/s eta 0:00:00


In [8]:
from langchain.document_loaders import PyPDFLoader

In [16]:
!wget https://raw.githubusercontent.com/manoharpavuluri/AI_FTCIntoTheDeep_RAGG/main/IntoTheDeep202425FTCCompetitionManualV7.pdf
loader = PyPDFLoader("IntoTheDeep202425FTCCompetitionManualV7.pdf")
pages = loader.load()
print(len(pages))

--2024-12-18 05:51:40--  https://raw.githubusercontent.com/manoharpavuluri/AI_FTCIntoTheDeep_RAGG/main/IntoTheDeep202425FTCCompetitionManualV7.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4478013 (4.3M) [application/octet-stream]
Saving to: ‘IntoTheDeep202425FTCCompetitionManualV7.pdf.1’

IntoTheDeep202425FT 100%[===================>]   4.27M  --.-KB/s    in 0.09s   

2024-12-18 05:51:40 (47.8 MB/s) - ‘IntoTheDeep202425FTCCompetitionManualV7.pdf.1’ saved [4478013/4478013]

144


In [17]:
full_text =""
for page in pages:
  full_text += page.page_content

print("Pages", len(pages))
print("Lines" , len(full_text.split("\n")))
print("Words" , len(full_text.split(" ")))
print("Charecters", len(full_text))

Pages 144
Lines 4895
Words 47851
Charecters 302498


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [66]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

393


In [68]:
print(chunks[10])

page_content='9.6 BASKETS .......................................................................................................................................... 52 
9.7 SCORING ELEMENTS ...................................................................................................................... 53 
9.7.1 SAMPLES .................................................................................................................................. 53 
9.7.2 CLIP ........................................................................................................................................... 54 
9.7.3 SPECIMEN ................................................................................................................................ 54 
9.8 AprilTags .......................................................................................................................................... 55' metadata={'source': 'IntoTheDeep202425FTCCompetitionManualV7.pdf', 'page': 2}


In [24]:
from langchain.embeddings import  OpenAIEmbeddings, CohereEmbeddings

In [26]:
#openAIEmbedding = OpenAIEmbeddings()
cohereEmbedding = CohereEmbeddings(user_agent="langchain")

<ipython-input-26-edd50d0d0199>:2: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereEmbeddings``.
  cohereEmbedding = CohereEmbeddings(user_agent="langchain")


In [27]:
!pip install langchain-cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 6.1 MB/s eta 0:00:00


In [28]:
from langchain_cohere import CohereEmbeddings


In [35]:
#openAIEmbedding = OpenAIEmbeddings()
cohereEmbed = CohereEmbeddings(model="embed-english-v3.0")

In [38]:
from langchain.vectorstores import Chroma



In [41]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00


In [69]:
vectorstore = Chroma.from_documents(persist_directory="/content/drive/MyDrive/Colab Notebooks/AI_FTCIntoTheDeep_RAG/AI_FTCIntoTheDeep_RAG_DB",
    documents=chunks,
    embedding=cohereEmbed,
)

In [70]:
vectorstore.persist()

In [71]:
vectorstore_retriever = vectorstore.as_retriever()


In [72]:
from langchain.tools.retriever import create_retriever_tool


In [73]:
vectorstore_search = create_retriever_tool(
    retriever=vectorstore_retriever,
    name="vectorstore_search",
    description="Retrieve relevant info from a vectorstore that contains documents related to FTC Challenge Into The Deep Competition Manual",
)

In [74]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.pydantic_v1 import BaseModel, Field

# Define the input schema for the search tool
class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")

    @classmethod
    def model_json_schema(cls):
        # Return the JSON schema of the model
        return cls.schema()

# Initialize the internet search tool
internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."
internet_search.args_schema = TavilySearchInput  # Assign the schema to the tool


In [75]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

# LLM
from langchain_cohere.chat_models import ChatCohere

chat = ChatCohere(model="command-r-plus", temperature=0.3)

# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource.
You are equipped with an internet search tool and a special vectorstore of information related to FTC Challenge Into The Deep Competition Manual.
If the query covers the topics of FTC Challenge Into The Deep Competition Manual then use the vectorstore search.
"""

# Prompt
prompt = ChatPromptTemplate.from_template("{input}")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=chat,
    tools=[internet_search, vectorstore_search],
    prompt=prompt,
)

In [76]:
# Agent Executor

agent_executor = AgentExecutor(
    agent=agent, tools=[internet_search, vectorstore_search], verbose=True
)

In [77]:
output = agent_executor.invoke(
    {
        "input": "What is the general election schedule of India 2024?",
        "preamble": preamble,
    }
)

print(output)

print(output['output'])



> Entering new AgentExecutor chain...

I will search for the general election schedule of India 2024 and write an answer based on what I find.
{'tool_name': 'internet_search', 'parameters': {'query': 'general election schedule of India 2024'}}
[{'url': 'https://www.reuters.com/graphics/INDIA-ELECTION/dwpkemxldpm/', 'content': "Schedule of 2024 India general elections. The elections in the world's largest democracy for 543 seats will be held in 7 phases. The chart shows India's election schedule over 7 phases with"}, {'url': 'https://www.india-briefing.com/news/india-2024-elections-parliament-seven-phases-31611.html/', 'content': "India's 2024 general election dates have been staggered from April 19 to June 1, per the Election Commission of India (ECI). In what is the world's largest electoral exercise, India's voter base of 970 million citizens, spanning 28 states and eight union territories, will vote under a seven-phase schedule.. The ECI will also conduct elections to the state le

In [81]:
output = agent_executor.invoke(
    {
        "input": "What are the robot constraints for the Into the Deep Robot?",
        "preamble": preamble,
    }
)

#print(output)

print(output['output'])




> Entering new AgentExecutor chain...

I will search the vector store for 'robot constraints' and write an answer based on the results.
{'tool_name': 'vectorstore_search', 'parameters': {'query': 'robot constraints'}}
Section 12 ROBOT Construction Rules (R) V7 92 of 144 
 
Teams must also be especially cautious when operating a running ROBOT during 
inspection, notifying the INSPECTOR that the ROBOT is live and taking every 
precaution to ensure the process is carried out safely. 
R103 *There is no ROBOT weight limit. There is no explicit weight limit for FIRST Tech Challenge ROBOTS. 
While there is no official weight limit, teams should still consider the impact of a 
ROBOT’S weight on various factors, including but not limited to:  
− FIELD TILE damage 
− battery consumption  
− ROBOT transportation 
− total ROBOT performance 
R104 There is a horizontal expansion limit. After the MATCH has started, ROBOTS may expand beyond the 
STARTING CONFIGURATION but are still subject to sizing

In [80]:
output = agent_executor.invoke(
    {
        "input": "Whats the Into the Deep Robot's horizontal expansion limit?",
        "preamble": preamble,
    }
)

#print(output)

print(output['output'])



> Entering new AgentExecutor chain...

I will search the vectorstore for the horizontal expansion limit of the Into the Deep Robot.
{'tool_name': 'vectorstore_search', 'parameters': {'query': "Into the Deep Robot's horizontal expansion limit"}}
Section 11 Game Rules (G) V7 83 of 144 
 
G418 Horizontal expansion limit. After the MATCH has started, ROBOTS may expand beyond the STARTING 
CONFIGURATION but are still subject to sizing constraints. The sizing constraints are specified in 
R104). 
Violation: if more than MOMENTARY, MINOR FOUL, or MAJOR FOUL if the over-expansion is used for 
strategic benefit, including if it impedes or enables a scoring action. 
This rule is intended to limit the amount of floor area each ROBOT can cover with 
the maximum range of motion of all extensions. All possible movement of 
extensions outside the STARTING CONFIGURATION must be constrained as 
described in R104.  
During the MATCH REFEREES may use ARENA elements to help gauge ROBOT 
expansion during